In [ ]:
%pip install torch

In [33]:
from pathlib import Path
import pandas as pd
import sys
import json
from importlib import reload
sys.path.append(".")
import baseline.baseline_functions as bl
import explore.truncate_and_template as tmp
reload(bl)
reload(tmp)

<module 'explore.truncate_and_template' from 'c:\\Users\\robin\\Documents\\projets\\severityPrediction\\src\\explore\\truncate_and_template.py'>

In [11]:
path_data = Path("../data/llm/data_preprocessed_tokens.json")

In [12]:
# Extract the relevant fields from the json file predictions_v100l.json
df = pd.read_json(path_data)
fields = ["bug_id","binary_severity","text","tokenized_text"]
df = df[fields]
df.rename({"text":"nltk_description","tokenized_text":"llama_tokenized_description"},axis=1,inplace=True)

In [34]:
# Get the template and apply preprocessing to get nltk template and llama2 tokenized version
template = tmp.get_template()
df_template = pd.DataFrame([{"description":template}])
nltk_template = bl.preprocess_text(df_template).iloc[0].description
data = [{"description":nltk_template}]
tmp.truncate_and_transform(data)
llama_tokenized_description = data[0]["truncated_token"]

Using pad_token, but it is not set yet.
1it [00:00, 31.08it/s]

Tokenizing {'description': 'Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Input:\n\n\n### Remembering the instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Response:'}


In [39]:
print(nltk_template)
print("*"*100)
print(llama_tokenized_description[101:])

Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Categorize the bug report into one of 2 categories:

0 = NOT SEVERE
1 = SEVERE

### Input:


### Remembering the instruction:
Categorize the bug report into one of 2 categories:

0 = NOT SEVERE
1 = SEVERE

### Response:
****************************************************************************************************
['<0x0A>', '<0x0A>', '<0x0A>', '##', '#', '▁Remember', 'ing', '▁the', '▁instruction', ':', '<0x0A>', 'C', 'ategor', 'ize', '▁the', '▁bug', '▁report', '▁into', '▁one', '▁of', '▁', '2', '▁categories', ':', '<0x0A>', '<0x0A>', '0', '▁=', '▁NOT', '▁SE', 'VER', 'E', '<0x0A>', '1', '▁=', '▁SE', 'VER', 'E', '<0x0A>', '<0x0A>', '##', '#', '▁Response', ':']


In [42]:
# build the final json
json_dict = {
    "template":{
        "nltk_template":nltk_template,
        "llama_tokenized_description":llama_tokenized_description,
        "description_index_insert": 101
    },
    "data":df.to_dict(orient="records")
}
with open("../data/llm/data_preprocessed_tokens_v2.json","w") as f:
    json.dump(json_dict,f,indent=2)